# US Public School Characteristics - 2020 - 21

#### Project Description

Starting off, and before I even found the datasets this analysis covers, I wanted to find a way to look at average teacher salary in the US and what factors affect salary the most for public school teachers across the country. While my analysis didn't end up covering that aspect as much as I originally intended, it did offer some surprising insights in regard to the vast differences students and teachers face when it comes to public schools in different locales, states, and regions in the US.

#### Insights

Overall, this notebook covers my process of working with the Public School Characteristics 2020-2021 dataset, and the massaging it took to get to the dataset I ended up analyzing. The finished article focuses primarily on school locations, student and teacher populations, student demographics, income-to-poverty ratio, and school locale.

In the end, this part of the project was most useful as an exercise in recording cleaning steps and process. As this was done individually, there are some ease-of-use features with excel that make filtering on multiple criteria a bit easier and faster than using Python, especially when investigating prior to cleaning. However, I included examples of how the process would be done using Python here.

# Data Cleaning 
Getting familiar with Public School Characteristics

To start, I imported the [Public School Characteristics csv file](https://data-nces.opendata.arcgis.com/datasets/nces::public-school-characteristics-current-1/explore?location=40.939600%2C-109.200589%2C8.89![image.png](attachment:image.png) in order to get an idea of what information was useful and could be removed. Fortunately, when I downloaded the dataset from the NCES database, there was a thorough summary of the data and descriptions for each of the columns. So while I needed to rename quite a few of the columns for ease of use, it was easy to verify what I was working with.

# Overview of steps
Overall, these are the steps that I went through to reach the finished dataset I worked with doing this project. 

1) Removed unused columns -> These are listed in the pertinent cell below, however there ended up being quite a few columns I did not use. In this workbook, I have removed the same columns as in my original work, however, some of the columns I thought I would need just didn't fit the finished project (such as Title I and Charter/Magnet School classification).

2) Renamed LSTATE -> STATE

3) Updated TOTAL column -> removed G13 and Adult Education schools, retotaled TOTAL column from PK - G12 columns for consistency.

4) Renamed and Rounded FTE -> Total_Teachers_Rounded
    4a) For mostly aesthetic reasons, I rounded the FTE column. This was brought on by two main factors, one being that I ended up needing to recalculate S:T Ratio for some schools/states due to odd values (-2, -1, 0, etc), and the other being that it made some of my visualizations easier to present without decimals for the number of teachers.
    
5) Renamed STUTERATIO -> Student_Teacher_Ratio and recalculated S:T Ratio for schools that were obviously wrong. For the sake of this project, I changed values based on reported Student and FTE numbers. In cases where that was not possible, I replaced blanks or incorrect values with the state reported S:T Ratio for the type of school I was dealing with. This happened on only a handful of records, so overall I don't think it made much difference and they probably could have been removed outright.

6) Renamed SY_STATUS_TEXT -> School_Status
    6a) Removed all schools that were not listed as "Currently Operational" or "Newly Opened" due to low impact on the dataset and general irrelevance to my topic. While I think there is value in looking at the closed schools or schools that are planning to reopen, I felt these were outside my scope and due to their low numbers (<1000), not worth keeping.

Finally, I also ended up making alternative tables and doing some additional calculations for percentage breakdowns for Demographics, IPR, Locales, and Teacher Salary. However, I think all of these things can be done in Tableau or simply as part of making plots, and I did not include them here. In the end, none of these calculations affected the dataset and did not result in any changes to the overall number of records.

In [1]:
# Import pandas and load the dataset
import pandas as pd

# Load CSV file
df = pd.read_csv("/Users/camerongomez/Desktop/Python Cleaning/Schools_Data_2021_2.csv")

In [2]:
# Print summary information - Get a feel for the data
print("Column names:")
print(list(df.columns))

print("Number of columns:", len(df.columns))
print("Number of rows:", len(df))

Column names:
['OBJECTID', 'X', 'Y', 'NCESSCH', 'SURVYEAR', 'STABR', 'LEAID', 'ST_LEAID', 'LEA_NAME', 'SCH_NAME', 'LSTREET1', 'LCITY', 'LSTATE', 'LZIP', 'PHONE', 'GSLO', 'GSHI', 'VIRTUAL', 'TOTFRL', 'FRELCH', 'REDLCH', 'PK', 'KG', 'G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08', 'G09', 'G10', 'G11', 'G12', 'G13', 'TOTAL', 'MEMBER', 'AM', 'HI', 'BL', 'WH', 'HP', 'TR', 'FTE', 'LATCOD', 'LONCOD', 'ULOCALE', 'NMCNTY', 'STUTERATIO', 'TITLEI', 'STITLEI', 'AMALM', 'AMALF', 'ASALM', 'ASALF', 'HIALM', 'HIALF', 'BLALM', 'BLALF', 'WHALM', 'WHALF', 'HPALM', 'HPALF', 'TRALM', 'TRALF', 'TOTMENROL', 'TOTFENROL', 'STATUS', 'UG', 'AE', 'SCHOOL_TYPE_TEXT', 'SY_STATUS_TEXT', 'SCHOOL_LEVEL', 'AS', 'CHARTER_TEXT', 'MAGNET_TEXT']
Number of columns: 77
Number of rows: 99742


In [3]:
# Get data type of each column
print("Data types:")
print(df.dtypes)

# Print the first 3 rows of each column
print("First 3 rows:")
for col in df.columns:
    print(col)
    print(df[col].head(3))
    print()


Data types:
OBJECTID            int64
X                 float64
Y                 float64
NCESSCH           float64
SURVYEAR           object
                   ...   
SY_STATUS_TEXT     object
SCHOOL_LEVEL       object
AS                  int64
CHARTER_TEXT       object
MAGNET_TEXT        object
Length: 77, dtype: object
First 3 rows:
OBJECTID
0    1
1    2
2    3
Name: OBJECTID, dtype: int64

X
0   -86.2062
1   -86.2049
2   -86.2201
Name: X, dtype: float64

Y
0    34.2602
1    34.2622
2    34.2733
Name: Y, dtype: float64

NCESSCH
0    1.000050e+10
1    1.000050e+10
2    1.000050e+10
Name: NCESSCH, dtype: float64

SURVYEAR
0    2020-2021
1    2020-2021
2    2020-2021
Name: SURVYEAR, dtype: object

STABR
0    AL
1    AL
2    AL
Name: STABR, dtype: object

LEAID
0    100005
1    100005
2    100005
Name: LEAID, dtype: int64

ST_LEAID
0    AL-101
1    AL-101
2    AL-101
Name: ST_LEAID, dtype: object

LEA_NAME
0    Albertville City
1    Albertville City
2    Albertville City
Name: LEA_NAME

At this point, I did some looking around after making a table in excel because I wanted to see some of the columns for myself. For simplicity, I did most of the next few steps directly in excel, but the code to complete the same steps using pandas is included below.

I used excel to quickly check what entries were in each column, and started considering what would or would not be useful given my final objective of making a story in Tableau. Overall, I found there were quite a few columns that I would have liked to use, but they didn't fit in nicely with what I was trying to accomplish, or the data was too imcomplete to use - such as male and female numbers of the included demographics.


In [4]:
# List of columns to remove - these are all unnecessary or incomplete
cols_to_remove = ['X', 'Y', 'STABR', 'LEAID', 'ST_LEAID', 'LEA_NAME', 'LZIP', 'G13', 'PHONE', 'AMALM', 'AMALF', 'ASALM', 'ASALF', 'HIALM', 'HIALF', 'BLALM', 'BLALF', 'WHALM', 'WHALF', 'HPALM', 'HPALF', 'TRALM', 'TRALF', 'TOTMENROL', 'TOTFENROL', 'UG', 'AE']

# Drop the columns
df.drop(cols_to_remove, axis=1, inplace=True)


After checking in excel again, I realized that the G13 column didn't have any necessary information, and the schools that ended up dropped in the end did not have any students in G13, so I also dropped that column as well.

In [ ]:
# Checking summary and table info again after removing the columns to check if there's anything I missed
print("Column names:")
print(list(df.columns))

print("Number of columns:", len(df.columns))
print("Number of rows:", len(df))

# Print the first 3 rows of each column
print("First 3 rows:")
for col in df.columns:
    print(col)
    print(df[col].head(3))

After removing some unnecessary columns, I wanted to start removing unnecessary records. The first column I looked at was "SY_STATUS_TEXT" where I wanted to keep schools that were currently open or newly opened, but did not want to keep any of the closed schools due to low numbers and low impact on the data set. Further, I did not anticipate needing to look at the results or impact of school closures for this project - although that would be worth investigating.

In [11]:
# Filter the data based on SY_STATUS_TEXT column
operational_data = df[df['SY_STATUS_TEXT'].str.strip().isin(['Currently operational', 'Newly opened'])]

# Count the number of removed records
removed_records = len(df) - len(operational_data)

# Count the number of removed records by LSTATE
removed_by_state = df[~df['SY_STATUS_TEXT'].str.strip().isin(['Currently operational', 'Newly opened'])]['LSTATE'].value_counts()

# Display the data without truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Display the filtered data and the counts
print('Number of removed records:', removed_records)
print('Number of removed records by state:\n', removed_by_state)
print('Operational data:')
print(operational_data)


Number of removed records: 2003
Number of removed records by state:
 TX    613
AZ    172
FL    110
NC     79
MN     59
WA     56
IL     55
ND     50
OH     48
NY     47
MI     44
UT     44
ID     41
NV     39
AR     37
WI     33
TN     29
SC     27
AL     26
SD     25
CO     24
NJ     24
NE     22
LA     22
OR     20
PA     19
MO     18
GA     18
KY     18
ME     17
OK     15
IN     14
CA     13
MS     13
IA     13
VA     12
AK     10
DC     10
MA      9
VT      8
MT      7
DE      7
WY      7
NM      6
KS      5
MD      4
WV      4
NH      4
HI      2
CT      2
RI      2
Name: LSTATE, dtype: int64
Operational data:


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
# Count the number of incomplete schools by state
incomplete_data = df[(df["FTE"] == 0) | (df["FTE"] == 1) 
                     & (df["STUTERATIO"] == -2) 
                     | (df["STUTERATIO"] == -1) 
                     | (df["STUTERATIO"] == 0) 
                     | (df["STUTERATIO"] == 1) 
                     & (df["MEMBER"] == 0) 
                     | (df["MEMBER"] == 1) 
                     & (df["SCHOOL_TYPE_TEXT"] == "Regular school") 
                     & (df["SY_STATUS_TEXT"].str.strip() == "Currently operational")].groupby("LSTATE").size().reset_index(name='counts')

# Add a total row
total = incomplete_data["counts"].sum()
incomplete_data.loc["Total"] = ["", total]

# Sort the output by counts, from largest to smallest
incomplete_data = incomplete_data.sort_values("counts", ascending=False)

# Print the output
print(incomplete_data)

      LSTATE  counts
Total          11533
12        IL    4384
42        UT    1113
3         AZ    1054
41        TX     790
7         FL     548
21        MN     392
4         CA     389
45        WA     299
43        VA     268
1         AL     228
20        MI     182
22        MO     176
23        MS     161
15        KY     155
32        NY     133
33        OH     132
29        NJ     101
27        NE      92
31        NV      83
40        TN      70
25        NC      69
38        SC      67
46        WI      66
47        WV      65
13        IN      57
26        ND      46
18        MD      44
11        ID      39
16        LA      38
19        ME      34
2         AR      30
39        SD      28
8         GA      26
36        PA      23
5         CO      20
30        NM      17
44        VT      16
34        OK      12
6         DE      12
0         AK      12
35        OR      11
37        RI       8
14        KS       8
17        MA       8
24        MT       8
48        WY 

At this point, I did a bit more digging into each column and checked some variations of schools with unexpected totals for Students, Teachers (FTE), and S:T Ratio.

For the first test, I looked at schools that were listed as "Currently Operational" and had Total_Teachers_Rounded to either 1 or 0. To narrow things down more, I included schools that had S:T Ratio of 0, and at least 1 Student.

To address the states I adjusted heavily - Utah, Arizona, and Illinois - that were otherwise blank or largely incomplete, to the extent that they were either wholly or almost entirely excluded from the analysis. Likely worth explaining that in a different setting, I would probably approach this issue differently, but my goal here was a complete dataset, rather than moving forward without the full set of states - especially when a simple alternative was readily available.

In [ ]:
# Filter the DataFrame to only include records where "FTE" is 0 and "SY_STATUS_TEXT" is "Currently operational"
incomplete_data = df[(df["FTE"] == 0) & (df["SY_STATUS_TEXT"] == "Currently operational")]

# Count the number of records per state in the filtered DataFrame
state_count = incomplete_data["LSTATE"].value_counts()

# Sort the counts by state abbreviation
state_counts = state_counts.sort_index()

# Print the state counts
print(state_counts)

Below, I'm going to check unique values in the 'LSTATE' column. I'll end up removing this, but interesting detail I noticed (after ages working with this data), the State Abbr column includes a unique abbr for schools on Indian Reservations. So even though the schools are in different states, there was a way to group by reservations as a whole. This could've been an avenue toward looking at the relatively grim outlook facing Am. Indian/Native American students. 

In [ ]:
# Count the number of distinct values in the "LSTATE" column
state_counts = df["LSTATE"].value_counts()

# Print the state counts
print(state_counts)

df.to_csv("/Users/camerongomez/Desktop/Python Cleaning/School_Test.csv", index=False)

After removing columns and incomplete records, I decided to join my two main csv files, Public School Characteristics and IPR/Poverty Statistics. I realized that I didn't actually need to do a join with Teacher Salary, so I left that for Tableau.

In the end, I used merge rather than join because it ended up being a little easier to match the columns on school name, rather than index number, due to the different order. However, this did take some massaging. To make sure that the join was giving me the desired results, I used two separate columns and checked the results against each other (the columns used were "SCH_NAME" and "NCESSCH" - the school name column seemed to give the desired results, but NCESSCH is a unique school ID number which removed any possible issues due to white space)

In [ ]:
# Load the IPR_Current.csv file
ipr_df = pd.read_csv("/Users/camerongomez/Desktop/Python Cleaning/IPR_Current.csv")

# Inner join the two DataFrames on the SCH_NAME column
df = pd.merge(df, ipr_df, on="SCH_NAME", how="inner")

# Print the number of rows in the merged DataFrame
print("Number of rows after merge:", len(df))


In [ ]:
# Save the merged DataFrame to a CSV file
df.to_csv("/Users/camerongomez/Desktop/Python Cleaning/School_Join_Final.csv", index=False)


In [ ]:
# Remove duplicates based on the "SCH_NAME" column
df.drop_duplicates(subset="SCH_NAME", keep="first", inplace=True)
# Save the merged DataFrame to a CSV file
df.to_csv("/Users/camerongomez/Desktop/Python Cleaning/School_Join_Final.csv", index=False)

# Conclusion
At this point, I brought both of my csv files into Tableau to start making plots and investigating using charts, which ended up bringing me to making some additional tables and calculations for percentages and averages related to Demographics, Locales, and IPR. As mentioned at the beginning of this notebook, I did not include those steps here, however, you can see them as part of the finished Tableau workbook found here: https://public.tableau.com/views/Capstone2_Public_Schools_US_Draft/Story-PublicSchoolCharacteristics?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link